# Práctico Graph Neural Network (GraphSAGE)


Sistemas Recomendadores <br>
**Profesor:** Denis Parra <br>
**Ayudantes:** Carlos Muñoz & Pablo Messina & Alejandro Plaza & Daniel Sebastian

Agradecimientos: Álvaro Labarca

En este práctico se utilizará un [tutorial interactivo de ArangoDB Interactive Tutorials](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/example_output/Comprehensive_GraphSage_Guide_with_PyTorchGeometric_Output.ipynb) para estudiar el funcionamiento del algoritmo GraphSAGE. GraphSAGE es un modelo de GCN (Graph Convolutional Network) que utiliza una red convolucional sobre datos representados en forma de grafos para generar embeddings que capturen las relaciones complejas entre distintos nodos. En particular, en este práctico se trabajará con el dataset [obgn-products](https://ogb.stanford.edu/docs/nodeprop/#ogbn-products), el cual utiliza un grafo homogéneo no direccionado para representar una cadena de co-compras de productos de Amazon, la cual será utilizada para generar embeddings de los productos que contiene.

## Important Note!!
If you are running this notebook on Google Colab, please make sure to enable hardware acceleration using either a GPU or a TPU. If it is run with CPU-only enabled, generating the word embeddings will take an incredibly long time! Hardware acceleration can be enabled by navigating to `Runtime` -> `Change Runtime`. This will present you with a popup, where you can select an appropriate `Hardware Accelerator`.

In [ ]:
import time
inicio = time.time()

In [ ]:
# Installing Pytorch Geometric
%%capture
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install -q torch-geometric
!pip install ogb
!pip install umap-learn

In [ ]:

import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
import os.path as osp
import pandas as pd
import numpy as np
import collections
from pandas.core.common import flatten
# importing obg datatset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from pandas.core.common import flatten
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(16.7,8.27)})
sns.set_theme(style="ticks")
import collections
from scipy.special import softmax
import umap

In [ ]:
# download and loading the obg dataset
root = osp.join(osp.dirname(osp.realpath('./')), 'data', 'products')
dataset = PygNodePropPredDataset('ogbn-products', root)

In [ ]:
# split_idx contains a dictionary of train, validation and test node indices
split_idx = dataset.get_idx_split()
# predefined ogb evaluator method used for validation of predictions
evaluator = Evaluator(name='ogbn-products')

Lets check the training, validation and test node split.

In [ ]:
# lets check the node ids distribution of train, test and val
print('Number of training nodes:', split_idx['train'].size(0))
print('Number of validation nodes:', split_idx['valid'].size(0))
print('Number of test nodes:', split_idx['test'].size(0))

In [ ]:
# loading the dataset
data = dataset[0]

Graph Statistics of the dataset

In [ ]:
# lets check some graph statistics of ogb-product graph
print("Number of nodes in the graph:", data.num_nodes)
print("Number of edges in the graph:", data.num_edges)
print("Node feature matrix with shape:", data.x.shape) # [num_nodes, num_node_features]
print("Graph connectivity in COO format with shape:", data.edge_index.shape) # [2, num_edges]
print("Target to train against :", data.y.shape)
print("Node feature length", dataset.num_features)

In [ ]:
# checking the number of unique labels
# there are 47 unique categories of product
data.y.unique()

In [ ]:
# load integer to real product category from label mapping provided inside the dataset
df = pd.read_csv('/data/products/ogbn_products/mapping/labelidx2productcategory.csv.gz')

In [ ]:
# lets see some of the product categories
df[:10]

In [ ]:
# creating a dictionary of product category and corresponding integer label
label_idx, prod_cat = df.iloc[: ,0].values, df.iloc[: ,1].values
label_mapping = dict(zip(label_idx, prod_cat))

In [ ]:
# counting the numbers of samples for each category
y = data.y.tolist()
y = list(flatten(y))
count_y = collections.Counter(y)
print(count_y)

## Neighborhood Sampling

This module iteratively samples neighbors (at each layer) and constructs bipartite graphs that simulate the actual computation flow of GNNs.

sizes: denotes how much neighbors we want to sample for each node in each layer.

`NeighborSampler` holds the current
    :obj:`batch_size`, the IDs :obj:`n_id` of all nodes involved in the
    computation, and a list of bipartite graph objects via the tuple
    :obj:`(edge_index, e_id, size)`, where :obj:`edge_index` represents the
    bipartite edges between source and target nodes, :obj:`e_id` denotes the
    IDs of original edges in the full graph, and :obj:`size` holds the shape
    of the bipartite graph.

The actual computation graphs are then returned in reverse-mode, meaning
    that we pass messages from a larger set of nodes to a smaller one, until we
    reach the nodes for which we originally wanted to compute embeddings.

To refer in detail: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/data/sampler.html

In [ ]:
train_idx = split_idx['train']
train_loader = NeighborSampler(data.edge_index, node_idx=train_idx,
                               sizes=[15, 10, 5], batch_size=1024,
                               shuffle=True)

In [ ]:
print(train_loader.edge_index)
print(train_idx)

# GraphSage Algorithm

In [ ]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3):
        super(SAGE, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.
        for i, (edge_index, _, size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
            xs.append(x)
            if i == 0:
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        #return x.log_softmax(dim=-1)
        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x)

                pbar.update(batch_size)

            if i == 0:
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all

        pbar.close()

        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGE(dataset.num_features, 256, dataset.num_classes, num_layers=3)
model = model.to(device)

In [ ]:
# loading node feature matrix and node labels
x = data.x.to(device)
y = data.y.squeeze().to(device)

In [ ]:
def train(epoch):
    model.train()

    #pbar = tqdm(total=train_idx.size(0))
    #pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]
        optimizer.zero_grad()
        l1_emb, l2_emb, l3_emb = model(x[n_id], adjs)
        #print("Layer 1 embeddings", l1_emb.shape)
        #print("Layer 2 embeddings", l1_emb.shape)
        out = l3_emb.log_softmax(dim=-1)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        #pbar.update(batch_size)

    #pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / train_idx.size(0)

    return loss, approx_acc

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

for epoch in range(1, 21):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')

In [ ]:
model

In [ ]:
#compute the number of trainable parameters:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_parameter = count_parameters(model)
print(total_parameter)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

## Saving the model for inference part

We need to save the model for the infernce part because google colab cannot create two graph loaders at the same time because of the limitation of the RAM size. Therefore, we first train with train_loader and then make inferences on test data using this saved model.

Here you can either save the model on google MyDrive or locally on your computer.


In [ ]:
#torch.save(model, '/content/drive/MyDrive/model_weights/graph_embeddings/model.pt')

# saving model in mydrive
from google.colab import drive
drive.mount('/content/drive')
fp = '/content/drive/MyDrive/model.pt'

torch.save(model, './model.pt')
torch.save(model, fp)

# Inference: Let's check GraphSage Inductive Power!!


This part includes making the use of trained GraphSage model in order to compute node embeddings and performing node category prediction on test data.
Aftwerwards, we compare the <b>U-Map visualizations of node embeddings</b> at 3 different layers of GraphSage and draw some interesting observations.




## Headsup : At this point of time you need to restart the colab runtime!!!


In [ ]:
# Installing Pytorch Geometric
%%capture
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install -q torch-geometric
!pip install ogb
!pip install umap-learn

## load the data


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
import os.path as osp
import pandas as pd
import numpy as np
import collections
from pandas.core.common import flatten
# importing obg datatset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from pandas.core.common import flatten
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(16.7,8.27)})
sns.set_theme(style="ticks")
import collections
from scipy.special import softmax
import umap

In [ ]:
# download and loading the obg dataset
root = osp.join(osp.dirname(osp.realpath('./')), 'data', 'products')
dataset = PygNodePropPredDataset('ogbn-products', root)

In [ ]:
# split_idx contains a dictionary of train, validation and test node indices
split_idx = dataset.get_idx_split()
# predefined ogb evaluator method used for validation of predictions
evaluator = Evaluator(name='ogbn-products')

In [ ]:
# loading the dataset
data = dataset[0]

In [ ]:
subgraph_loader = NeighborSampler(data.edge_index, node_idx=None, sizes=[-1],
                                  batch_size=1024, shuffle=False)

In [ ]:
# load integer to real product category from label mapping provided inside the dataset
df = pd.read_csv('/data/products/ogbn_products/mapping/labelidx2productcategory.csv.gz')

In [ ]:
# creating a dictionary of product category and corresponding integer label
label_idx, prod_cat = df.iloc[: ,0].values, df.iloc[: ,1].values
label_mapping = dict(zip(label_idx, prod_cat))
print("Label mapping", label_mapping)

In [ ]:
y = data.y.tolist()
y = list(flatten(y))
count_y = collections.Counter(y)
print(count_y)

## Need to define the model class again

In [ ]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3):
        super(SAGE, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.
        for i, (edge_index, _, size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
            xs.append(x)
            if i == 0:
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        #return x.log_softmax(dim=-1)
        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x)

                pbar.update(batch_size)

            if i == 0:
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all

        pbar.close()

        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

In [ ]:
# loading the saved model
# Load model from Drive
from google.colab import drive
drive.mount('/content/drive')
fp = '/content/drive/My Drive/model.pt'
model = torch.load(fp)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# load node feature matrix and labels
x = data.x.to(device)
y = data.y.squeeze().to(device)

Perform evaluation on test data with saved model

In [ ]:
@torch.no_grad()
def test():
    model.eval()

    l1_embeddings, l2_embeddings, l3_embeddings = model.inference(x)
    out = l3_embeddings
    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    test_acc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return test_acc, l1_embeddings, l2_embeddings, l3_embeddings

In [ ]:
# shapes
test_acc, l1_embeddings, l2_embeddings, l3_embeddings = test()

In [ ]:
print('Final Test acc:', test_acc)

## Embeddings of whole datastet layer-wise

In [ ]:
print("Layer 1 Embeddings Shape:",l1_embeddings.shape)
print("Layer 2 Embeddings Shape:",l2_embeddings.shape)
print("Layer 3 Embeddings Shape:",l3_embeddings.shape)

## Embeddings of Test data layer-wise

Delete x and y variables with the original data entries in order to make room for the RAM in Colab.

In [ ]:
del x
del y

In [ ]:
l1_embedding_test = l1_embeddings[split_idx['test']]
l2_embedding_test = l2_embeddings[split_idx['test']]
l3_embedding_test = l3_embeddings[split_idx['test']]

In [ ]:
l1_embedding_test.shape, l2_embedding_test.shape, l3_embedding_test.shape

In [ ]:
l3_embedding_smax_test = softmax(l3_embedding_test.detach().cpu().numpy(), axis=1)
l3_embedding_smax_test.shape

In [ ]:
# saving predictions from last layer embeddings
y_pred = np.argmax(l3_embedding_smax_test, axis=1)
y_pred.shape

## Embedding Visualizations with Umap

In [ ]:
reducer = umap.UMAP()

## Layer-1 Node Embeddings Visualization

In [ ]:
# sample test data
l1_emb_sample = l1_embedding_test[:2000].detach().cpu().numpy()
y_pred_sample = y_pred[:2000]
l1_emb_sample.shape, y_pred_sample.shape

In [ ]:
y_pred_sample

In [ ]:
# label mapping
y_pred_sample_products = []
for y in y_pred_sample:
    y_pred_sample_products.append(label_mapping[y])

y_pred_sample_products = np.array(y_pred_sample_products )

In [ ]:
# number of unique classes present in sampled data
len(set(y_pred_sample_products.tolist()))

27

In [ ]:
color_coding = ['green','yellowgreen','brown','dodgerblue','red','black', 'grey',
                'darkgreen', 'cyan', 'yellow', 'magenta', 'lightcoral', 'rosybrown', 'maroon',
                'peru', 'khaki', 'olive', 'darkseagreen', 'lightseagreen', 'lightskyblue', 'darkviolet',
                'pink', 'deeppink', 'thistle', 'ivory', 'gold', 'lavender']

Now we need to train our reducer, letting it learn about the manifold. For this UMAP follows the sklearn API and has a method fit which we pass the data we want the model to learn from.



In [ ]:
l1_reduced_emb = reducer.fit_transform(l1_emb_sample)

In [ ]:
# reduced representation
l1_reduced_emb.shape

In [ ]:
y_pred_sample_products.shape

In [ ]:
# plot
sns.scatterplot(x = l1_reduced_emb[:, 0], y = l1_reduced_emb[:, 1], hue = y_pred_sample_products, palette=color_coding)
# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

1. Comente la capacidad de la primera capa del modelo para representar las clases, ¿Qué clases representa bien?, ¿Cuáles no representa bien? y ¿Por qué cree usted?

In [ ]:
# Respuesta

## Layer-2 Node Embeddings Visualization

In [ ]:
# sample test data
l2_emb_sample = l2_embedding_test[:2000].detach().cpu().numpy()
y_pred_sample = y_pred[:2000]
l2_emb_sample.shape, y_pred_sample.shape

In [ ]:
l2_reduced_emb = reducer.fit_transform(l2_emb_sample)

In [ ]:
# plot
sns.scatterplot(x = l2_reduced_emb[:, 0], y = l2_reduced_emb[:, 1], hue = y_pred_sample_products,  palette=color_coding)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

2. Comente la capacidad de la segunda capa del modelo para representar las clases, ¿Qué clases representa bien?, ¿Cuáles no representa bien? y ¿Por qué cree usted?. Comente los resultados en relación a la capa anterior.

In [ ]:
# Respuesta

## Layer-3 Node Embeddings Visualization

In [ ]:
# sample test data
l3_emb_sample = l3_embedding_test[:2000].detach().cpu().numpy()
y_pred_sample = y_pred[:2000]
l3_emb_sample.shape, y_pred_sample.shape

In [ ]:
l3_reduced_emb = reducer.fit_transform(l3_emb_sample)

In [ ]:
# plot
sns.scatterplot(x = l3_reduced_emb[:, 0], y = l3_reduced_emb[:, 1], hue = y_pred_sample_products, palette=color_coding)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

3. Comente la capacidad de la tercera capa del modelo para representar las clases, ¿Qué clases representa bien?, ¿Cuáles no representa bien? y ¿Por qué cree usted?. Comente los resultados en relación a las capas anteriores.

In [ ]:
# Respuesta

4. Comente dos casos de uso que pueden verse beneficiados al usar las representaciones anteriores.

In [ ]:
# Respuesta

In [ ]:
end_time = time.time()
execution_time = end_time - start_time

print(f"Tiempo de ejecución: {execution_time} segundos")

Tiempo de ejecución: 14034.83057641983 segundos
